In this notebook I'm updating my simpler attention function to incorporate masking, and also doing some more thorough testing of it. Later I'll need to ensure that the parallelized attention function is working and also implement masking/appropriate testing for that.

In [1]:
import torch
from torch import tensor
from torch.nn.functional import softmax
from math import sqrt

In order for masking to work, I need to know the position of the current token (query token) relative to the positions of all the key/value tokens. This information might actually be more accessible in the parallelized versions since query/key/value matrices might have the same indexing relative to the original input (meaning for example that the 5th value vector and 5th query vector are both associated with the 5th input token). I think in this case I need to manually pass in the position of the query token, as an index of the key/value vectors. So passing in position 5 would mean that the query token also has the value vector on row 5 of the values matrix, and I can therefore mask every row of the values matrix after that. In the parallelized function I think this will look more like cutting out a diagonal slice of the scaled weights matrix.

In [2]:
# I assume all rows have the same dimension, which is provided in dim
# keys, values are assumed to be structured such that each key/value is a row.
# Of course, there must also be an equal number of keys and values, so those matrices must have equal dimensions
from numpy import Infinity


def attention(query: tensor, keys: tensor, values: tensor, dim: int, query_pos: int) -> tensor:
    # query = query.view(1, -1)
    raw_weights = query @ keys.T

    # Masking:
    for i in range(query_pos + 1, len(raw_weights)):
        raw_weights[i] = -1 * Infinity

    scale_factor = sqrt(dim)
    scaled_weights = softmax(raw_weights / scale_factor, dim=0)

    scaled_values = scaled_weights.view(-1, 1) * values
    contextualized_value = torch.sum(scaled_values, 0)

    return contextualized_value


Simple test case that I did by hand. Expected result is approximately [3, 2], which is in fact what we see!

In [3]:
Q = tensor([1, 3]).float()
K = tensor([[2, 1], [3, 5]]).float()
V = tensor([[0, 5], [3, 2]]).float()
d = 2
print(attention(Q, K, V, d, 1))
print(attention(Q, K, V, d, 0))

tensor([2.9997, 2.0003])
tensor([0., 5.])


Now I'll do the same thing with my parallel attention function, although I won't worry about adding masking quite yet.

In [4]:
def par_attention(queries: tensor, keys: tensor, values: tensor, dim: int) -> tensor:
    raw_weights = queries @ keys.T
    # for query_pos in range(0, queries.shape[0]):
    #     for weight_pos in range(query_pos + 1, dim):
    #         raw_weights[query_pos][weight_pos] = -1 * Infinity
    mask = torch.tril(torch.ones_like(raw_weights), diagonal=0)
    raw_weights = raw_weights.masked_fill(mask == 0, float('-inf'))
    print(raw_weights)

    scale_factor = sqrt(dim)
    scaled_weights = softmax(raw_weights / scale_factor, dim=1)

    # now scaled weights is a matrix where each row represents the scaled weights produced based on a given query.
    # meanwhile values just has a value vector on each row.

    reshaped_scaled_weights = scaled_weights.view(scaled_weights.shape[0], scaled_weights.shape[1], 1)
    reshaped_values = values.view(1, values.shape[0], values.shape[1])

    scaled_values = reshaped_scaled_weights * reshaped_values

    contextualized_values = torch.sum(scaled_values, 1)
    return contextualized_values

In [5]:
Q = tensor([[1, 3], [1, 1]]).float()
K = tensor([[2, 1], [3, 5]]).float()
V = tensor([[0, 5], [3, 2]]).float()
d = 2
par_attention(Q, K, V, d)


tensor([[5., -inf],
        [3., 8.]])


tensor([[0.0000, 5.0000],
        [2.9150, 2.0850]])

That looks like it's working right to me, which is great! I now have masking implemented and a much more fully validated, parallel attention function.

Is the function differentiable?

In [6]:
# Define toy input tensors (ensure that they require gradients)
queries = torch.randn(5, 4, requires_grad=True)
keys = torch.randn(5, 4, requires_grad=True)
values = torch.randn(5, 4, requires_grad=True)

# Call your attention function
output = par_attention(queries, keys, values, 4)

# Sum up the elements of the output to get a scalar
# This is a simple stand-in for a loss function
loss = output.sum()

# Backpropagate
loss.backward()

# Check the gradients
print("Gradients w.r.t queries:", queries.grad)
print("Gradients w.r.t keys:", keys.grad)
print("Gradients w.r.t values:", values.grad)
print("output:", output)


tensor([[ 1.8960,    -inf,    -inf,    -inf,    -inf],
        [-1.2210, -0.7865,    -inf,    -inf,    -inf],
        [ 0.3465,  0.2114,  2.7768,    -inf,    -inf],
        [-1.8242, -3.2753,  1.4139, -3.8097,    -inf],
        [-2.3554,  0.6812, -1.5110,  5.3552,  5.0509]],
       grad_fn=<MaskedFillBackward0>)
Gradients w.r.t queries: tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0050,  0.0037,  0.0016, -0.0039],
        [ 0.0951, -0.0160,  0.0157,  0.0757],
        [ 0.2160,  0.0423,  0.1490,  0.1625],
        [ 0.3444, -0.0057,  0.1117,  0.0539]])
Gradients w.r.t keys: tensor([[-0.0375, -0.0254, -0.0587, -0.0281],
        [-0.0511, -0.0898, -0.1118, -0.0287],
        [ 0.1658,  0.0013,  0.0208,  0.1151],
        [-0.0192,  0.3194,  0.2776, -0.0713],
        [-0.0580, -0.2056, -0.1279,  0.0130]])
Gradients w.r.t values: tensor([[1.7898, 1.7898, 1.7898, 1.7898],
        [0.8485, 0.8485, 0.8485, 0.8485],
        [1.3827, 1.3827, 1.3827, 1.3827],
        [0.5515, 0.5515, 0.

I've edited the previous function a bit, starting to add support for batch processing which I think is necessary. Gonna get it fully working here before bringing it back to transformer-arch notebook.

In [19]:
def par_attention(queries: tensor, keys: tensor, values: tensor, dim: int) -> tensor:
    raw_weights = torch.bmm(queries, keys.transpose(1, 2))

    mask = torch.tril(torch.ones_like(raw_weights), diagonal=0)
    raw_weights = raw_weights.masked_fill(mask == 0, float('-inf'))
    print(f"raw_weights.shape:{raw_weights.shape}\nraw_weights: {raw_weights}")

    scale_factor = sqrt(dim)
    scaled_weights = softmax(raw_weights / scale_factor, dim=2)
    print(f"scaled_weights.shape:{scaled_weights.shape}\nscaled_weights: {scaled_weights}")

    # now scaled weights is a matrix where each row represents the scaled weights produced based on a given query.
    # meanwhile values just has a value vector on each row.

    reshaped_scaled_weights = scaled_weights.view(scaled_weights.shape[0], scaled_weights.shape[1], scaled_weights.shape[2], 1)
    reshaped_values = values.view(1, values.shape[0], values.shape[1], values.shape[2])

    scaled_values = reshaped_scaled_weights * reshaped_values

    contextualized_values = torch.sum(scaled_values, 2)
    return contextualized_values

In [20]:
Q = tensor([[1, 3], [1, 1]]).float().unsqueeze(0)
K = tensor([[2, 1], [3, 5]]).float().unsqueeze(0)
V = tensor([[0, 5], [3, 2]]).float().unsqueeze(0)
d = 2
par_attention(Q, K, V, d)


raw_weights.shape:torch.Size([1, 2, 2])
raw_weights: tensor([[[5., -inf],
         [3., 8.]]])
scaled_weights.shape:torch.Size([1, 2, 2])
scaled_weights: tensor([[[1.0000, 0.0000],
         [0.0283, 0.9717]]])


tensor([[[0.0000, 5.0000],
         [2.9150, 2.0850]]])